# Bibliotecas

In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [5]:
import numpy as np

# Criando um Navegador 

In [7]:
service = Service()

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=service, options=options)

# Navegando até o site do DataSUS

Entrar na Página 

In [8]:
url = 'http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/spabr.def'

driver.get(url)

In [9]:
lin = ""
arqs = []
colunas_df = ['Município', 
           '01 Ações de promoção e prevenção em saúde',
           '02 Procedimentos com finalidade diagnóstica',
           '03 Procedimentos clínicos',
           '04 Procedimentos cirúrgicos',
           '05 Transplantes de orgãos, tecidos e células',
           '06 Medicamentos',
           '07 Órteses, próteses e materiais especiais',
           '08 Ações complementares da atenção à saúde',
           'QTD TOTAL']


# Em "Coluna" Selecionar a opção "Grupo Procedimento"
driver.find_element(By.XPATH, '//*[@id="C"]/option[7]').click()

# Em "Conteúdo" Deselecionar "Quantidade Aprovada" 
driver.find_element(By.XPATH, '//*[@id="I"]/option[1]').click()

# Em "Conteúdo" Selecionar "Valor Aprovado" 
driver.find_element(By.XPATH, '//*[@id="I"]/option[2]').click()

# Deselecionar o Periodo
driver.find_element(By.XPATH, f'//*[@id="A"]/option[1]').click()

for periodo in range(193):   
    
    # Selecionar Periodo
    driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').click()
    data = driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').text + ','
    print('Coletando dados de', data)
    
    # Selecionar 'Colunas Separadas por ";"'
    driver.find_element(By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[1]/div[2]/input[3]').click()
    
    # Clicar em Mostrar
    driver.find_element(By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()
    
    # Ir até nova aba
    janelas_abertas = driver.window_handles
    driver.switch_to.window(janelas_abertas[1])
    
    print('Gerando e Salvando Dataframe')
    # Gerar e Salvar Dataframe
    linhas = driver.find_element(By.XPATH, '/html/body/div/div/pre').text.split('\n')
    colunas = linhas[0].split(';')
    dados = [linha.split(';') for linha in linhas[1:]]
    df = pd.DataFrame(dados, columns=colunas)
    df['Data'] = data
    arqs.append(df)
    
    # Fechar Aba atual e navegar até primeira aba
    driver.close()
    janelas_abertas = driver.window_handles
    driver.switch_to.window(janelas_abertas[0])    
    
    # Deselecionar Periodo
    driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').click()
    
#Concatenar arquivos
df_resultado = pd.concat(arqs, ignore_index=True)

# Exportar Arquivo
df_resultado.to_csv('procedimentos_datasus_16_anos_valor.csv', index=False)

Coletando dados de Jan/2024,
Gerando e Salvando Dataframe
Coletando dados de Dez/2023,
Gerando e Salvando Dataframe
Coletando dados de Nov/2023,
Gerando e Salvando Dataframe
Coletando dados de Out/2023,
Gerando e Salvando Dataframe
Coletando dados de Set/2023,
Gerando e Salvando Dataframe
Coletando dados de Ago/2023,
Gerando e Salvando Dataframe
Coletando dados de Jul/2023,
Gerando e Salvando Dataframe
Coletando dados de Jun/2023,
Gerando e Salvando Dataframe
Coletando dados de Mai/2023,
Gerando e Salvando Dataframe
Coletando dados de Abr/2023,
Gerando e Salvando Dataframe
Coletando dados de Mar/2023,
Gerando e Salvando Dataframe
Coletando dados de Fev/2023,
Gerando e Salvando Dataframe
Coletando dados de Jan/2023,
Gerando e Salvando Dataframe
Coletando dados de Dez/2022,
Gerando e Salvando Dataframe
Coletando dados de Nov/2022,
Gerando e Salvando Dataframe
Coletando dados de Out/2022,
Gerando e Salvando Dataframe
Coletando dados de Set/2022,
Gerando e Salvando Dataframe
Coletando dado

# Tratar Dados

In [18]:
# Gerar Dataframe com Dados Históricos
df = pd.read_csv('procedimentos_datasus_16_anos_valor.csv', sep=',', quotechar='"')

In [19]:
df.head()

,"""Município""","""02 Procedimentos com finalidade diagnóstica""","""03 Procedimentos clínicos""","""04 Procedimentos cirúrgicos""","""05 Transplantes de orgãos, tecidos e células""","""06 Medicamentos""","""07 Órteses, próteses e materiais especiais""","""08 Ações complementares da atenção à saúde""","""Total""",Data
0,"""110001 ALTA FLORESTA D'OESTE""","238,15","45595,51","434,99",-,-,-,"1312,00","47580,65","Jan/2024,"
1,"""110002 ARIQUEMES""","505,65","13549,24",-,-,"55,00",-,"100800,00","114909,89","Jan/2024,"
2,"""110004 CACOAL""","45702,65","272620,27","517434,68","9005,89","1236,40","46785,09","494275,55","1387060,53","Jan/2024,"
3,"""110006 COLORADO DO OESTE""",-,"15974,94","615,60",-,-,-,"1088,95","17679,49","Jan/2024,"
4,"""110009 ESPIGAO D'OESTE""","1,00","26360,51","1036,80",-,-,-,"1800,00","29198,31","Jan/2024,"


In [20]:
# Remover Aspas do Nome das Colunas
df.columns = [coluna.replace('"', '') for coluna in df.columns]

# Remover Aspas do Campo das Colunas
df['Município'] = df['Município'].str.replace('"', '')
df['02 Procedimentos com finalidade diagnóstica'] = df['02 Procedimentos com finalidade diagnóstica'].replace("-", '0')
df['03 Procedimentos clínicos'] = df['03 Procedimentos clínicos'].replace("-", '0')
df['04 Procedimentos cirúrgicos'] = df['04 Procedimentos cirúrgicos'].replace("-", '0')
df['05 Transplantes de orgãos, tecidos e células'] = df['05 Transplantes de orgãos, tecidos e células'].replace("-", '0')
df['06 Medicamentos'] = df['06 Medicamentos'].replace("-", '0')
df['07 Órteses, próteses e materiais especiais'] = df['07 Órteses, próteses e materiais especiais'].replace("-", '0')
df['08 Ações complementares da atenção à saúde'] = df['08 Ações complementares da atenção à saúde'].replace("-", '0')
df['Total'] = df['Total'].replace("-", '0')

# Criar Coluna com Codigo e Nome do Municipio
df['Cod_Município'] = df['Município'].str.slice(0, 6)
df['Município'] = df['Município'].str.slice(6)

# Criar e Tratar Colunas com Variáveis "Mês" e "Ano"
df[['Mês', 'Ano']] = df["Data"].str.split('/', expand=True)
df['Ano'] = df['Ano'].apply(lambda x: x.replace(',', ''))
df = df.drop('Data', axis=1)

# Renomear coluna "Total" por "Valor Total"
df = df.rename(columns={'Total': 'Valor Total'})

# Reordenar Dataframe
df = df[['Ano','Mês','Cod_Município','Município',
       '02 Procedimentos com finalidade diagnóstica',
       '03 Procedimentos clínicos', '04 Procedimentos cirúrgicos',
       '04 Procedimentos cirúrgicos',
       '05 Transplantes de orgãos, tecidos e células', '06 Medicamentos',
       '07 Órteses, próteses e materiais especiais',
       '08 Ações complementares da atenção à saúde', 'Valor Total']]

# Salvar Dataframe
df.to_csv('procedimentos_datasus_16_anos_valor_tratados.csv', index=False, decimal='.')

In [21]:
df.head()

,Ano,Mês,Cod_Município,Município,02 Procedimentos com finalidade diagnóstica,03 Procedimentos clínicos,04 Procedimentos cirúrgicos,04 Procedimentos cirúrgicos,"05 Transplantes de orgãos, tecidos e células",06 Medicamentos,"07 Órteses, próteses e materiais especiais",08 Ações complementares da atenção à saúde,Valor Total
0,2024,Jan,110001,ALTA FLORESTA D'OESTE,"238,15","45595,51","434,99","434,99",0,0,0,"1312,00","47580,65"
1,2024,Jan,110002,ARIQUEMES,"505,65","13549,24",0,0,0,"55,00",0,"100800,00","114909,89"
2,2024,Jan,110004,CACOAL,"45702,65","272620,27","517434,68","517434,68","9005,89","1236,40","46785,09","494275,55","1387060,53"
3,2024,Jan,110006,COLORADO DO OESTE,0,"15974,94","615,60","615,60",0,0,0,"1088,95","17679,49"
4,2024,Jan,110009,ESPIGAO D'OESTE,"1,00","26360,51","1036,80","1036,80",0,0,0,"1800,00","29198,31"
